<a href="https://colab.research.google.com/github/whopriyam/Benchmarking-Transformer-Networks-for-Sexual-Harassment-Detection-on-Twitter/blob/main/Models/BigBird_Roberta_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install simpletransformers
# Optional: for Experiment management
!pip install wandb

     |████████████████████████████████| 225kB 33.9MB/s 
     |████████████████████████████████| 3.3MB 45.8MB/s 
     |████████████████████████████████| 1.8MB 43.2MB/s 
     |████████████████████████████████| 2.3MB 41.6MB/s 
     |████████████████████████████████| 81kB 11.3MB/s 
     |████████████████████████████████| 8.2MB 33.9MB/s 
     |████████████████████████████████| 122kB 60.2MB/s 
     |████████████████████████████████| 51kB 8.4MB/s 
     |████████████████████████████████| 235kB 55.5MB/s 
     |████████████████████████████████| 1.2MB 35.0MB/s 
     |████████████████████████████████| 102kB 14.8MB/s 
     |████████████████████████████████| 133kB 39.9MB/s 
     |████████████████████████████████| 174kB 60.0MB/s 
     |████████████████████████████████| 901kB 41.9MB/s 
     |████████████████████████████████| 4.2MB 37.7MB/s 
     |████████████████████████████████| 81kB 10.9MB/s 
     |████████████████████████████████| 112kB 59.2MB/s 
     |████████████████████████████████| 245kB 48.1MB

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/text classification/Harassment_Cleaned_tweets.csv')

In [3]:
df = df.filter(['Text','Label'], axis=1)
df

,Text,Label
0,He’d have my phone wedged up his ass sideways.,1
1,Yep 100% agree and the same with severine in s...,0
2,Stop having sex with men that won’t eat your p...,1
3,"When you see a sad man, just give him breast t...",1
4,We can't be together if you don't eat pussy,1
...,...,...
3599,only for a 5'7 brown hair brown eye gamer girl...,1
3600,"Don't be suspicious! Some butt, some suggestiv...",0
3601,mhmmmmm i hear ya big tittie lil booty,1
3602,I hate dirty stank ass cowboy fans bro lmao,0


In [4]:
train_df = df.sample(frac=0.7)
test_df = df.loc[~df.index.isin(train_df.index)]


train_final = train_df.sample(frac = 0.8)
eval_df = train_df.loc[~train_df.index.isin(train_final.index)]
len(eval_df), len(train_df) , len(test_df)

(505, 2523, 1081)

In [5]:
import pandas as pd
import pytest

from simpletransformers.classification import (
    ClassificationModel,
    # MultiLabelClassificationModel,
)


model_type = "bigbird"
model_name = "google/bigbird-roberta-base"


In [7]:
 model = ClassificationModel(
        model_type,
        model_name,
        use_cuda=False,
        args={
            "no_save": True,
            "reprocess_input_data": True,
            "overwrite_output_dir": True,
            "scheduler": "constant_schedule",
            "max_seq_length": 20,
        },
    )

Some weights of the model checkpoint at google/bigbird-roberta-base were not used when initializing BigBirdForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BigBirdForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BigBirdForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BigBirdForSequenceClassifica

In [8]:
# Train the model
model.train_model(train_df)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:602: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/2523 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/316 [00:00<?, ?it/s]

Attention type 'block_sparse' is not possible if sequence_length: 20 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3.Changing attention type to 'original_full'...


(316, 0.6162371200191069)

In [10]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test_df)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1384: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/1081 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/136 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [16]:
list_eval = eval_df['Text'].to_list()
type(list_eval)

list

In [20]:
predictions, raw_outputs = model.predict(list_eval)

  0%|          | 0/505 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

In [21]:
from sklearn import metrics


In [22]:
targets = eval_df['Label'].to_list()
metrics.accuracy_score(targets, predictions)

0.49306930693069306

In [23]:
print(metrics.classification_report(targets, predictions))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       256
           1       0.49      1.00      0.66       249

    accuracy                           0.49       505
   macro avg       0.25      0.50      0.33       505
weighted avg       0.24      0.49      0.33       505



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
